In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader

from src.core.spice import circuits, spice_utils, subcircuits, xyce
from src.data.xor_datamodule import XORDataModule, XORwithBiasDataModule

In [ ]:
ckpt_path = "/home/minsik/ml/logs/train/runs/2024-10-22_22-50-45/checkpoints/epoch_036.ckpt"
ckpt_path2 = "/home/minsik/ml/logs/train/runs/2024-10-24_10-56-07/checkpoints/epoch_202.ckpt"
ckpt_30K_2M = "/home/minsik/ml/logs/train/runs/2024-10-25_17-20-13/checkpoints/epoch_180.ckpt"
checkpoint = torch.load(ckpt_30K_2M)
# Extract weights from the checkpoint (assuming net.model layers)
W1 = checkpoint["state_dict"]["net.model.0.weight"]  # First layer weights
W2 = checkpoint["state_dict"]["net.model.1.weight"]  # Second layer weights

# Combine them into a list (if needed)
W = [W1, W2]

# Print the extracted weights
print("W1 (First Layer Weights):", W1)
print("W2 (Second Layer Weights):", W2)

In [ ]:
print(W1)
print(W2)

In [ ]:
data_module = XORwithBiasDataModule(batch_size=1)

data_module.setup()

# Create a DataLoader
dataloader = DataLoader(
    data_module.dataset, batch_size=data_module.batch_size, num_workers=data_module.num_workers
)

# Iterate through the DataLoader to extract batches of x and y
for batch in dataloader:
    x, y = batch  # x will contain the inputs, y will contain the targets

    # If you want to access individual elements from the batch
    for i in range(len(x)):
        x_item = x[i]
        y_item = y[i]
        print(f"x: {x_item}, y: {y_item}")

In [ ]:
dims = [2, 2]
SPICE_params = {
    "A": 1,
    "beta": "0.001",
    "Diode": {
        "Path": "/home/minsik/ml/src/core/spice/1N4148.lib",
        "ModelName": "1N4148",
        "Rectifier": "BidRectifier",
    },
    "noise": 0,
}

# Create the PyCircuit
Pycircuit = circuits.create_circuit(
    input=x[0][0][0],  # Assuming x is already defined
    bias=None,
    W=W,  # Loaded from the checkpoint
    dimensions=dims,
    **SPICE_params,
)

# Copy from PyCircuit to ShallowCircuit
circuit = circuits.ShallowCircuit.copyFromCircuit(Pycircuit)

In [ ]:
spice_utils.SPICENNParser.updateWeight(circuit, W)

In [ ]:
print(circuit)

In [ ]:
sim = xyce.XyceSim()

nodes_list = []
for batch in dataloader:
    x, y = batch  # x will contain the inputs, y will contain the targets

    # If you want to access individual elements from the batch
    for i in range(len(x)):
        print(x[i][0][0])
        spice_utils.SPICENNParser.clampLayer(circuit, x[i][0][0])
        raw_file = sim(spice_input=circuit)
        voltages = spice_utils.SPICENNParser.fastRawfileParser(
            raw_file, nodenames=circuit.nodes, dimensions=dims
        )

        combined_voltages = np.concatenate([voltages[1][0], voltages[1][1]])
        nodes_list.append(combined_voltages)

In [ ]:
voltages